<a href="https://colab.research.google.com/github/Carnoustie/DeepLearningClassKTH/blob/main/gradeE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from tensorflow.keras import datasets, layers, models, regularizers, preprocessing, optimizers
from tensorflow.keras.optimizers import SGD
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [9]:
(trainX,trainLabels), (testX,testLabels) = datasets.cifar10.load_data()

#Map pixel values from [0, 255}  to -> [0, 1]
#trainX, testX = trainX/255.0 , testX/255.0


trainMean = np.mean(trainX,0)
trainDev = np.std(trainX,0)
trainX = trainX-trainMean
trainX = trainX/trainDev

testX = trainX-trainMean
testX = trainX/trainDev


print("Sanity Check:\n\nmean ", np.mean(trainX),  "\n\nvar: ", np.std(trainX))


Sanity Check:

mean  7.94919685631612e-18 

var:  0.9999999999999998


In [10]:
trainLabels = tf.keras.utils.to_categorical(trainLabels)
testLabels = tf.keras.utils.to_categorical(testLabels)




In [13]:


model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_uniform',padding='same', input_shape=(32,32,3)))
model.add(layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.2))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(10, activation='softmax'))






In [12]:
LearningRate = optimizers.schedules.CosineDecay(0.005, 100*trainX.shape[0],0.002)
opt = SGD(LearningRate, momentum=0.9)
#model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
#history = model.fit(trainX, trainLabels, epochs=10, validation_data=(testX, testLabels))

generator = preprocessing.image.ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

batches = generator.flow(trainX, trainLabels, batch_size=64)

model.compile(optimizer=opt,
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

history = model.fit(batches, steps_per_epoch=trainX.shape[0]/64,epochs=100,
                    validation_data=(testX, testLabels))

Epoch 1/100
 14/781 [..............................] - ETA: 7:14 - loss: 2.8576 - accuracy: 0.1194

KeyboardInterrupt: ignored

In [ ]:
model.evaluate(testX,testLabels,verbose=0)

In [ ]:
#plt.plot(history.history['accuracy'], label='accuracy')
#plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
#plt.xlabel('Epoch')
#plt.ylabel('Accuracy')
#plt.ylim([0.5, 1])
#plt.legend(loc='lower right')

#test_loss, test_acc = model.evaluate(testX,  testLabels, verbose=2)

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch iteration')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.savefig('regularize3.png')
testLoss, testAccuracy = model.evaluate(testX,testLabels, verbose=2)